In [10]:
data = '/opt/data/public/retail/'

In [16]:
import sklearn
from sklearn import tree
from sklearn import neural_network
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [12]:
sklearn.tree.DecisionTreeClassifier

sklearn.tree.tree.DecisionTreeClassifier

## 生成训练集和测试集

In [20]:
cancer_data = load_breast_cancer()
print(cancer_data)
x_train, x_test, y_train, y_test = train_test_split(cancer_data['data'], cancer_data['target'], test_size = 0.7)

## 自编决策树

## sklearn中决策树

In [23]:
sklearn_decision_tree = tree.DecisionTreeClassifier()
sklearn_decision_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [25]:
sklearn_decision_tree.score(x_test, y_test)

0.899749373433584

In [40]:
import os
import pydot

files_dir_path = './files'
if not os.path.exists(files_dir_path):
    os.mkdir(files_dir_path)
sklearn_tree_dot_file = 'tree.dot'
sklearn_tree_dot_file_path = os.path.join(files_dir_path, sklearn_tree_dot_file)
if True or not os.path.exists(sklearn_tree_dot_file_path):
    tree.export_graphviz(sklearn_decision_tree, out_file=sklearn_tree_dot_file_path, class_names=['严重','轻微'],feature_names=cancer_data.feature_names,impurity=False,filled=True)
(graph, ) = pydot.graph_from_dot_file(sklearn_tree_dot_file_path)

sklearn_tree_png_file = 'tree.png'
sklearn_tree_png_file_path = os.path.join(files_dir_path, sklearn_tree_png_file)
graph.write_png(sklearn_tree_png_file_path)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe8 in position 148: ordinal not in range(128)